#Trajectory SAPT Analysis
###Imports

In [1]:
import MDAnalysis as mda
import psi4
import matplotlib.pyplot as plt

###Defining SAPT calculator function

In [ ]:
def calculate_spat(selections: list, protein: mda.Universe):
    # Writes xyz coords from trajectory
    def write_xyz(selection, universe):
        group = universe.select_atoms('resid')
        items = selection.split()
        name = ''
        for item in items:
            if item != 'and' or item != 'resid' or item != 'name':
                name += '_' + item
        with mda.Writer(( name + '.xyz'), group.n_atoms) as coords:
            coords.write(group)
        return

    # Gets xyz data from saved coords
    def read_xyz(selection):
        path = selection + '.xyz'
        xyz_data = []
        with open(path, 'r') as coord_file:
            coord_data = coord_file.readlines()[1:]
            images = 0
            for line in coord_data:
                if line == '\n':
                    images += 1

            for line in coord_data:
                if '.' in line:
                    xyz_data.append(line)
        return xyz_data

    # Saves coords as a string
    def save_coords(coords0, coords1):
        coord_data = '0 1\n'

        for line in coords0:
            coord_data += line

        coord_data += '--\n'
        coord_data += '0 1\n'

        for line in coords1:
            coord_data += line

        coord_data += 'units angstrom'
        return coord_data


    write_xyz(selections[0], protein)
    write_xyz(selections[1], protein)

    group0_coords = read_xyz(selections[0])
    group1_coords = read_xyz(selections[1])

    psi_in = save_coords(group0_coords, group1_coords)
    dimer = psi4.geometry(psi_in)

    psi4.set_options({'scf_type': 'df',
                      'freeze_core': 'true'})
    energy = psi4.energy('spat0/jun-cc-pvdz', molecule=dimer)
    return energy

###Creates plot from SAPT calculation

In [ ]:
if __name__ == '__main__':
    # Test files
    topology = 'testfiles/testtop.psf'
    trajectory = 'testfiles/testtraj.dcd'

    test_unv = mda.Universe(topology, trajectory)
    time_list = []
    energies = []

    for ts in test_unv.trajectory:
        time_list.append(ts)
        energies.append(calculate_spat([1, 2], test_unv))


    plt.plot(time_list, energies, label='Energy')
    plt.xlabel('Time(ns)')
    plt.ylabel('Energy')
    plt.savefig('SAPT.png', dpi=1200)
